1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [3]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2023-11-16 10:07:08--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2023-11-16 10:07:09--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6cb111206f8024929c5e38b20b.dl.dropboxusercontent.com/cd/0/inline/CHoJ_PD3dQ5KPKMW1n2wfdLwuWhYFoYFw02XERgF0klKIwRiagI6K5qfBcpDO3jCnw9QnYXMv2Jn5WxFpGQ9gMTdjqE_daIo8pZXMY5iWbtnAM1wBVaezF8cTU53ICOGBM8/file# [following]
--2023-11-16 10:07:10--  https://uc6cb111206f8024929c5e38b20b.dl.dropboxusercontent.com/cd/0/inline/CHoJ_PD3dQ5KPKMW1n2wfdLwuWhYFoYFw02XERgF0klKIwRiagI6K5qfBcpDO3jCnw9QnYXMv2Jn5WxFpGQ9gMTdjqE_daIo8pZXMY5iWbtnAM1wBVaezF8cTU53ICOGBM8/file
Res

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [4]:
import pandas as pd # standard naming convention
import numpy as np

In [7]:
file_name = "./data/data_000637.txt"
N = 11000
data = pd.read_csv(file_name, nrows = N)
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
10995,1,0,139,3869200277,204,0
10996,1,0,61,3869200277,202,20
10997,1,0,63,3869200277,204,1
10998,1,0,59,3869200277,283,29


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [22]:
BX = data.loc[:,['BX_COUNTER']]
BX.sort_values(by="BX_COUNTER", ascending=False)
print("The number of BX in an ORBIT is", BX.iloc[0,0])

The number of BX in an ORBIT is 2374


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [125]:
dfa = data.copy()
dfa['abs_time_ns'] = dfa['ORBIT_CNT'] * dfa['BX_COUNTER'] * 25 + dfa['BX_COUNTER'] * 25 + dfa['TDC_MEAS']

abs_time = dfa.loc[:,['abs_time_ns']]
dfa['abs_time_ns'] = abs(abs_time.iloc[0,0] - dfa['abs_time_ns']) 

sr3 = pd.Series(dfa['abs_time_ns'], index=dfa.index)
print(sr3)


0                      0
1                      1
2         17314670751802
3         17798320772793
4         37337781621199
              ...       
10995    209904108553026
10996    210097568566906
10997    209904108553025
10998    202262438003947
10999    202455898017852
Name: abs_time_ns, Length: 11000, dtype: int64


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [132]:
duration = pd.to_datetime(sr3)

time = duration.max() - duration.min()

hours, remainder = divmod(time.total_seconds(), 3600)
minutes, seconds = divmod(remainder, 60)

print(f"This is the duration of the data: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds.")


This is the duration of the data: 63 hours, 47 minutes, 17 seconds.


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [79]:
df2 = data.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()

top = df2.sort_values(ascending=False).head(3)

print("These are the top three most noisy channels:")

print(top)


These are the top three most noisy channels:
TDC_CHANNEL
139    941
64     554
63     548
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [101]:
columns = data.iloc[:, [1,3]]
df_query = columns.query('FPGA > 0')
df3 = df_query.drop_duplicates(keep='first')
print('There are', len(df3), 'non-empty orbits.')

There are 110 non-empty orbits.


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [94]:
dfq = data.query('TDC_CHANNEL == 139')
dfl = dfq.iloc[:, 0:4]
df4 = dfl.drop_duplicates(keep=False)

print("There are", len(df4), "unique orbits with at least one measurement from TDC_CHANNEL = 139")

There are 19 unique orbits with at least one measurement from TDC_CHANNEL = 139


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [119]:
index_series = data['TDC_CHANNEL']
df81 = data.query('FPGA > 0')
df80 = data.query('FPGA <= 0')

fpga1 = df81.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()
fpga0 = df80.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()

sr1 = pd.Series(fpga1, index=fpga1.index)
sr0 = pd.Series(fpga0, index=fpga1.index)
print("This is the series for FPGA = 1 \n", sr1)
print("This is the series for FPGA = 0 \n", sr0)

This is the series for FPGA = 1 
 TDC_CHANNEL
1      253
2      275
3      212
4      220
5      146
      ... 
125      2
126      1
127     10
128     11
139    288
Name: TDC_CHANNEL, Length: 116, dtype: int64
This is the series for FPGA = 0 
 TDC_CHANNEL
1        5.0
2       10.0
3        9.0
4       12.0
5       13.0
       ...  
125      NaN
126      NaN
127      NaN
128      NaN
139    653.0
Name: TDC_CHANNEL, Length: 116, dtype: float64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.